### **For given aoi, run Deforestation model(Unet-Diff) inference**

In [1]:
import io
import json
import logging
import os
import re
import shutil
import tempfile
import urllib
import uuid
import warnings
from datetime import datetime, timedelta
import time
from os.path import join
from pathlib import Path

import cv2
import geojson
import geopandas
import geopandas as gp
import geopandas as gpd
import imageio
import numpy as np
import pandas as pd
import pyproj
import rasterio
import rasterio.mask
import segmentation_models_pytorch as smp
import shapely
import torch
import torch.backends.cudnn as cudnn
from geojson import Feature
from geopandas import GeoSeries
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from rasterio import Affine, features
from rasterio.mask import mask as riomask
from rasterio.merge import merge
from rasterio.plot import reshape_as_image, reshape_as_raster
from rasterio.warp import Resampling, calculate_default_transform, reproject
from rasterio.windows import Window
from scipy import spatial
from sentinel2download.downloader import Sentinel2Downloader, logger
from sentinel2download.overlap import Sentinel2Overlap
from shapely import wkt
from shapely.geometry import MultiPolygon, Polygon, box
from shapely.ops import transform, unary_union
from skimage.exposure import match_histograms
from torch import nn
from torchvision import transforms
from tqdm import tqdm
from rasterio.mask import mask as rasterio_mask
import shapely


warnings.filterwarnings("ignore")


# Functions to create, load the model(Use them instead of Catalyst lib code)
def prepare_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")


def prepare_model(model):
    device = prepare_device()

    if torch.cuda.is_available():
        cudnn.benchmark = True

    if torch.cuda.device_count() > 1:
        raise ValueError("Multi GPU mode is not supported")
    else:
        model = model.to(device)

    return model, device


def load_model(network, model_weights_path):
    if network == "unet-diff":
        aux_params = dict(
            pooling="max",  # one of 'avg', 'max'
            dropout=0.1,  # dropout ratio, default is None
            activation="sigmoid",  # activation function, default is None
            classes=1,  # define number of output labels
        )
        model = smp.Unet(
            "resnet18",
            aux_params=aux_params,
            encoder_weights=None,
            in_channels=27,
            encoder_depth=2,
            decoder_channels=(256, 128),
        )

    else:
        raise ValueError("Unknown network")

    model, device = prepare_model(model)
    model.load_state_dict(
        torch.load(model_weights_path, map_location=torch.device(device))
    )
    return model, device

### 0. Setting up parameters
#### Read Input from environment and setup folders and filenames

In [2]:
current_time = time.strftime("deforestation_cache_%Y_%m_%d_%H:%M:%S", time.gmtime())
REQUEST_ID = os.getenv("REQUEST_ID")
START_DATE = os.getenv("START_DATE")

END_DATE = os.getenv("END_DATE")
AOI = os.getenv("AOI")
SENTINEL2_GOOGLE_API_KEY = os.getenv("SENTINEL2_GOOGLE_API_KEY")
SATELLITE_CACHE_FOLDER = os.path.join(os.getenv("SENTINEL2_CACHE"), current_time)

INPUT_FOLDER = os.path.dirname(SATELLITE_CACHE_FOLDER)
OUTPUT_FOLDER = os.getenv("OUTPUT_FOLDER")
PREPARED_DATA_FOLDER = os.path.join(SATELLITE_CACHE_FOLDER, "prepared")
CODE_FOLDER = "/code"

LANDCOVER_POLYGONS_PATH = os.path.join(CODE_FOLDER, "data", "landcovers")
LANDCOVER_FILENAME = (
    "S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml"
)
SENTINEL_TILES = os.path.join(LANDCOVER_POLYGONS_PATH, LANDCOVER_FILENAME)

SCOPES = ["https://www.googleapis.com/auth/drive.file"]

CLOUD_DATA_FOLDER = os.path.join(CODE_FOLDER, "data", "clouds")
MODEL_PATH = os.path.join(CODE_FOLDER, "models", "unet_diff.pth")

LOAD_DIR = SATELLITE_CACHE_FOLDER

PRODUCT_TYPE = "L1C"
BANDS = {"TCI", "B01", "B02", "B04", "B05", "B08", "B8A", "B09", "B10", "B11", "B12"}
CONSTRAINTS = {
    "NODATA_PIXEL_PERCENTAGE": 15.0,
    "CLOUDY_PIXEL_PERCENTAGE": 15.0,
}

NEAREST_POLYGONS_NUMBER = 10
# DATES_FOR_TILE = 2
CLOUDS_PROBABILITY_THRESHOLD = 0.95
REMOVE_OTHER_DATES = True

NODATA = 0
MAX_SHIFT_ITERS = 2
MAX_SHIFT = 5

In [3]:
os.makedirs(INPUT_FOLDER, exist_ok=True)
os.makedirs(LANDCOVER_POLYGONS_PATH, exist_ok=True)
os.makedirs(PREPARED_DATA_FOLDER, exist_ok=True)
os.makedirs(CLOUD_DATA_FOLDER, exist_ok=True)
os.makedirs(SATELLITE_CACHE_FOLDER, exist_ok=True)

### 1. Transform AOI to GeoJSON file and then download data from Sentinel L1C

In [5]:
aoi = gp.GeoDataFrame(geometry=[wkt.loads(AOI)], crs="epsg:4326")
aoi_filename = "provided_aoi.geojson"
aoi.to_file(aoi_filename, driver="GeoJSON")

In [6]:
s2overlap = Sentinel2Overlap(aoi_path=aoi_filename)
overlap_tiles = s2overlap.overlap_with_geometry()

In [7]:
def shift_date(date, delta=5, format="%Y-%m-%d"):
    date = datetime.strptime(date, format)
    date = date - timedelta(days=delta)
    return datetime.strftime(date, format)


def diff_date(date_a, date_b, format="%Y-%m-%d"):
    date_a, date_b = datetime.strptime(date_a, format), datetime.strptime(
        date_b, format
    )
    delta = date_a - date_b
    return delta

In [8]:
start_timestamp, end_timestamp = datetime.strptime(
    START_DATE, "%Y-%m-%d"
), datetime.strptime(END_DATE, "%Y-%m-%d")
year = ((end_timestamp - start_timestamp) / 2 + start_timestamp).year
year

2019

In [9]:
def check_folder(folder, limit=15, nodata=0):
    for file in os.listdir(folder):
        if file.endswith(".jp2"):
            with rasterio.open(os.path.join(folder, file)) as src:
                data = src.read(1)
                file_nodata_percentage = (
                    round((data == nodata).sum() / data.size, 2) * 100
                )
            if file_nodata_percentage > limit:
                return False
            return True

In [10]:
def load_images(tiles, start_date, end_date):
    loader = Sentinel2Downloader(SENTINEL2_GOOGLE_API_KEY)
    loadings = dict()

    for tile in tiles:
        start = start_date
        end = end_date

        print(f"Loading images for tile: {tile}...")
        count = 0
        while count < MAX_SHIFT_ITERS:
            loaded = loader.download(
                PRODUCT_TYPE,
                [tile],
                start_date=start,
                end_date=end,
                output_dir=LOAD_DIR,
                bands=BANDS,
                constraints=CONSTRAINTS,
            )
            
            distinct_folders = set()
            for duo_files in loaded:
                distinct_folders.add(os.path.dirname(duo_files[0]))

            loaded_ = []
            for folder in distinct_folders:
                if len(os.listdir(folder)) >= len(BANDS) + 1: #skip folder if it didn't have all needed BANDS
                    if check_folder(
                        folder, limit=CONSTRAINTS["NODATA_PIXEL_PERCENTAGE"], nodata=NODATA
                    ): # found if folders has nodata
                        for duo_files in loaded:
                            if folder in duo_files[0]:
                                loaded_.append(duo_files)
            loaded = loaded_
            if not loaded:
                previous_end = end
                previous_start = start
                end = start
                start = shift_date(start, delta=MAX_SHIFT)
                print(
                    f"For tile: {tile} and dates {previous_start} {previous_end} proper images not found! Shift dates to {start} {end}!"
                )
            else:
                break
            count += 1
        if loaded:
            loadings[tile] = loaded
            print(f"Loading images for tile {tile} finished")
        else:
            print(f"Images for tile {tile} were not loaded!")

    # tile_folders = dict()
    # for tile, tile_paths in loadings.items():
    #    tile_folders[tile] = {str(Path(tile_path[0]).parent) for tile_path in tile_paths}
    return loadings

In [11]:
loadings_start_date = load_images(
    overlap_tiles.Name.values, shift_date(START_DATE, delta=10), START_DATE
)
loadings_end_date = load_images(
    overlap_tiles.Name.values, shift_date(END_DATE, delta=10), END_DATE
)

Loading images for tile: 21JXL...
For tile: 21JXL and dates 2019-03-22 2019-04-01 proper images not found! Shift dates to 2019-03-17 2019-03-22!
For tile: 21JXL and dates 2019-03-17 2019-03-22 proper images not found! Shift dates to 2019-03-12 2019-03-17!
Images for tile 21JXL were not loaded!
Loading images for tile: 21JXK...
Loading images for tile 21JXK finished
Loading images for tile: 21JYL...
Loading images for tile 21JYL finished
Loading images for tile: 21JYK...
Loading images for tile 21JYK finished
Loading images for tile: 21JXL...
For tile: 21JXL and dates 2020-03-22 2020-04-01 proper images not found! Shift dates to 2020-03-17 2020-03-22!
For tile: 21JXL and dates 2020-03-17 2020-03-22 proper images not found! Shift dates to 2020-03-12 2020-03-17!
Images for tile 21JXL were not loaded!
Loading images for tile: 21JXK...
Loading images for tile 21JXK finished
Loading images for tile: 21JYL...
Loading images for tile 21JYL finished
Loading images for tile: 21JYK...
Loading ima

In [12]:
if not loadings_start_date or not loadings_end_date:  
    raise ValueError("Images not loaded for given AOI. Change dates, constraints")

In [13]:
def filter_by_date(loadings, func=max, filtered=dict(), tag="start"):
    def _find_agg_date(folders, func=func):
        dates = list()
        for i, folder in enumerate(folders):
            search = re.search(r"_(\d+)T\d+_", str(folder))
            date = search.group(1)
            date = datetime.strptime(date, "%Y%m%d")
            dates.append(date)
        last_date = func(dates)
        last_date = datetime.strftime(last_date, "%Y%m%d")
        return last_date

    def _get_folder(files, last_date):
        for idx, duo in enumerate(files):
            if last_date in duo[0]:
                return os.path.dirname(duo[0])
        raise Exception("last_date folder not found")

    for tile, items in loadings.items():
        try:
            last_date = _find_agg_date(items)
            bands_paths = dict()
            for path, _ in items:
                if PRODUCT_TYPE == "L2A":
                    if last_date in path:
                        if "B8A_20m.jp2" in path:
                            bands_paths["B8A"] = path
                        if "B11_20m.jp2" in path:
                            bands_paths["B11"] = path
                        if "B04_10m.jp2" in path:
                            bands_paths["B04"] = path
                        if "B08_10m.jp2" in path:
                            bands_paths["B08"] = path
                        if "B12_20m.jp2" in path:
                            bands_paths["B12"] = path
                        if "TCI_10m.jp2" in path:
                            bands_paths["TCI"] = path
                        folder = _get_folder(items, last_date)
                elif PRODUCT_TYPE == "L1C":
                    if last_date in path:
                        if "B8A.jp2" in path:
                            bands_paths["B8A"] = path
                        if "B11.jp2" in path:
                            bands_paths["B11"] = path
                        if "B04.jp2" in path:
                            bands_paths["B04"] = path
                        if "B08.jp2" in path:
                            bands_paths["B08"] = path
                        if "B12.jp2" in path:
                            bands_paths["B12"] = path
                        if "TCI.jp2" in path:
                            bands_paths["TCI"] = path
                        folder_path = _get_folder(items, last_date)

            info_dict = {
                tag: dict(paths=bands_paths, date=last_date, folder=folder_path)
            }

            if tile in filtered.keys():
                filtered[tile].update(info_dict)
            else:
                filtered.update({tile: info_dict})

        except Exception as ex:
            print(f"Error for {tile}: {str(ex)}")
    return filtered

In [14]:
filtered = filter_by_date(loadings_start_date, func=max, tag="start")
filtered = filter_by_date(loadings_end_date, func=max, filtered=filtered, tag="end")
filtered

{'21JXK': {'start': {'paths': {'B04': '/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20190327T134209_N0207_R124_T21JXK_20190327T202148/T21JXK_20190327T134209_B04.jp2',
    'TCI': '/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20190327T134209_N0207_R124_T21JXK_20190327T202148/T21JXK_20190327T134209_TCI.jp2',
    'B11': '/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20190327T134209_N0207_R124_T21JXK_20190327T202148/T21JXK_20190327T134209_B11.jp2',
    'B8A': '/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20190327T134209_N0207_R124_T21JXK_20190327T202148/T21JXK_20190327T134209_B8A.jp2',
    'B12': '/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20190327T134209_N0207_R124_T21JXK_20190327T202148/T21JXK_20190327T134209_B12.jp2',
    'B08': '/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20190327T134209_N0207_R124_T21JXK_20190327

In [15]:
def get_tiles_folders_dates(fitered):
    tiles = tuple(fitered.keys())
    start_date_folders = [filtered[tile]["start"]["folder"] for tile in tiles]
    end_date_folders = [filtered[tile]["end"]["folder"] for tile in tiles]
    dates = [filtered[tiles[0]]['start']['date'], filtered[tiles[0]]['end']['date']]
    return tiles, start_date_folders, end_date_folders, dates


tiles, start_date_folders, end_date_folders, dates = get_tiles_folders_dates(filtered)
assert len(start_date_folders) == len(end_date_folders), 'Not enough tiles folders'
start_date_folders, end_date_folders, tiles, dates

(['/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20190327T134209_N0207_R124_T21JXK_20190327T202148',
  '/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20190327T134209_N0207_R124_T21JYL_20190327T202148',
  '/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20190327T134209_N0207_R124_T21JYK_20190327T202148'],
 ['/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2A_MSIL1C_20200326T134211_N0209_R124_T21JXK_20200326T152251',
  '/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20200331T134209_N0209_R124_T21JYL_20200331T165744',
  '/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20200331T134209_N0209_R124_T21JYK_20200331T165744'],
 ('21JXK', '21JYL', '21JYK'),
 ['20190327', '20200326'])

In [16]:
import shutil


def remove_not_used_dates(
    start_date_folder, end_date_folder, cache_dir=SATELLITE_CACHE_FOLDER
):
    start_split, end_split = start_date_folder.split("/"), end_date_folder.split("/")
    if cache_dir != os.path.join("/", *start_split[:-1]) or cache_dir != os.path.join(
        "/", *end_split[:-1]
    ):
        raise ValueError("cache_dir is not valid")
    used_dates = start_split[-1], end_split[-1]
    for folder in os.listdir(cache_dir):
        if folder not in used_dates:
            shutil.rmtree(os.path.join(cache_dir, folder))


# if REMOVE_OTHER_DATES:
#     remove_not_used_dates(start_date_folder, end_date_folder)

### 2. Preparing images (calculating ndmi ndvi, scaling, merging to tiff)

In [17]:
def crop_aoi_from_raster(raster_path, aoi_gdf, output_path):
    with rasterio.open(raster_path, 'r') as src:
        meta = src.meta
        aoi_gdf = aoi_gdf.to_crs(meta['crs'])
        shapes_to_crop = [shapely.ops.unary_union(aoi_gdf.geometry.to_list())]
        mask, out_trans = rasterio_mask(src,
                                        shapes_to_crop,
                                        nodata=0,
                                        crop=True)

        meta['height'] = mask.shape[1]
        meta['width'] = mask.shape[2]
        meta['transform'] = out_trans


    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(mask)
    print(f'saved raster {output_path}')
    os.remove(raster_path)
    print(f'removed raster {raster_path}')
    return output_path

In [18]:
from rasterio.merge import merge


def transform_crs(data_path, save_path, dst_crs="EPSG:4326", resolution=(10, 10)):
    with rasterio.open(data_path) as src:
        if resolution is None:
            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds
            )
        else:
            transform, width, height = calculate_default_transform(
                src.crs,
                dst_crs,
                src.width,
                src.height,
                *src.bounds,
                resolution=resolution,
            )
        kwargs = src.meta.copy()
        kwargs.update(
            {"crs": dst_crs, "transform": transform, "width": width, "height": height}
        )
        with rasterio.open(save_path, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest,
                )

    return save_path


def stitch_tiles(paths, out_raster_path='test.tif'):
    tiles = []
    tmp_files = []
    
    for i, path in enumerate(paths):
        if i == 0:
            file = rasterio.open(path)
            meta, crs = file.meta, file.crs
        else:
            tmp_path = path.replace(
                '.jp2', '_tmp.jp2').replace('.tif', '_tmp.tif')
            crs_transformed = transform_crs(path, tmp_path, 
                                            dst_crs=crs, 
                                            resolution=None)
            tmp_files.append(crs_transformed)
            file = rasterio.open(crs_transformed)
        tiles.append(file)
            
    tile_arr, transform = rasterio.merge.merge(tiles, method='last')
    
    
    meta.update({"driver": "GTiff",
                 "height": tile_arr.shape[1],
                 "width": tile_arr.shape[2],
                 "transform": transform,
                 "crs": crs})
    
    if '.jp2' in out_raster_path:
        out_raster_path = out_raster_path.replace('.jp2', '_merged.tif')
    else:
        out_raster_path = out_raster_path.replace('.tif', '_merged.tif')
    print(f'saved raster {out_raster_path}')

    for tile in tiles:
        tile.close()
        
    for tmp_file in tmp_files:
        try:
            os.remove(tmp_file)
        except FileNotFoundError:
            print(f'Tile {tmp_file} was removed or renamed, skipping')
        
    with rasterio.open(out_raster_path, "w", **meta) as dst:
        dst.write(tile_arr)
    
    return out_raster_path

In [19]:
from time_dependent.data_prepare.prepare_tif import (get_ndmi, get_ndvi, merge,
                                                     scale_img, search_band,
                                                     to_tiff)


def prepare_data(data_folder, save_path):
    img_folder = data_folder

    tmp_file = os.path.basename(data_folder)

    os.makedirs(save_path, exist_ok=True)
    save_file_merged = join(save_path, f"all_merged_{tmp_file}.tif")
    print(save_file_merged)
    bands, band_names = ["TCI", "B08", "B8A", "B11", "B12"], []

    for band in bands:
        band_names.append(join(img_folder, search_band(band, img_folder, "jp2")))

    b4_name = join(img_folder, search_band("B04", img_folder, "jp2"))
    ndvi_name = join(img_folder, "ndvi")
    ndmi_name = join(img_folder, "ndmi")
    print("\nall bands are converting to *tif...\n")

    for band_name in band_names:
        print(band_name[-3:])
        if "B08" in band_name:
            b8_name = band_name
        if "B8A" in band_name:
            b8a_name = band_name
        if "B11" in band_name:
            b11_name = band_name
        to_tiff(f"{band_name}.jp2")

    to_tiff(f"{b4_name}.jp2")
    print("\nndvi band is processing...")

    get_ndvi(f"{b4_name}.tif", f"{b8_name}.tif", f"{ndvi_name}.tif")

    print("\nndmi band is processing...")

    get_ndmi(f"{b11_name}.tif", f"{b8a_name}.tif", f"{ndmi_name}.tif")

    band_names.append(ndvi_name)
    band_names.append(ndmi_name)

    bands.append("ndvi")
    bands.append("ndmi")

    print("\nall bands are scaling to 8-bit images...\n")
    band_names_scaled = []
    for band_name in band_names:
        print(band_name)
        scaled_name = scale_img(f"{band_name}.tif")
        band_names_scaled.append(scaled_name)

    print("\nall bands are being merged...\n")
    print(band_names_scaled)

    merge(save_file_merged, *band_names_scaled)

    for item in os.listdir(img_folder):
        if item.endswith(".tif"):
            os.remove(join(img_folder, item))
    return save_file_merged


In [20]:
tiles_prefix = "".join(tiles)
tiles_prefix

'21JXK21JYL21JYK'

In [21]:
os.makedirs(PREPARED_DATA_FOLDER, exist_ok=True)


start_merged = []
end_merged = []

for start_date_folder, end_date_folder in zip(start_date_folders, end_date_folders):
    
    start_date_merged_path = prepare_data(start_date_folder, PREPARED_DATA_FOLDER)
    end_date_merged_path = prepare_data(end_date_folder, PREPARED_DATA_FOLDER)
    
    start_merged.append(start_date_merged_path)
    end_merged.append(end_date_merged_path)

start_date_merged_path = stitch_tiles(start_merged, out_raster_path=os.path.join(PREPARED_DATA_FOLDER, f'all_merged_{dates[0]}_date_{tiles_prefix}.tif'))
end_date_merged_path = stitch_tiles(end_merged, out_raster_path=os.path.join(PREPARED_DATA_FOLDER, f'all_merged_{dates[1]}_date_{tiles_prefix}.tif'))

    

/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/prepared/all_merged_S2B_MSIL1C_20190327T134209_N0207_R124_T21JXK_20190327T202148.tif

all bands are converting to *tif...

TCI
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B08
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B8A
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
B11
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
B12
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.

ndvi band is processing...
0 .. 10 .. 20 .. 30 .. 40 .. 50 .. 60 .. 70 .. 80 .. 90 .. 100 - Done

ndmi band is processing...
0 .. 10 .. 20 .. 30 .. 40 .. 50 .. 60 .. 70 .. 80 .. 90 .. 100 - Done

all bands are scaling to 8-bit images...

/in

Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2A_MSIL1C_20200326T134211_N0209_R124_T21JXK_20200326T152251/T21JXK_20200326T134211_B11
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2A_MSIL1C_20200326T134211_N0209_R124_T21JXK_20200326T152251/T21JXK_20200326T134211_B12
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2A_MSIL1C_20200326T134211_N0209_R124_T21JXK_20200326T152251/ndvi
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2A_MSIL1C_20200326T134211_N0209_R124_T21JXK_20200326T152251/ndmi
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...


Processing file     1 of     7,  0.000% completed in 0 minutes.
Filename: /input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20190327T134209_N0207_R124_T21JYL_20190327T202148/T21JYL_20190327T134209_TCI_scaled.tif
File Size: 10980x10980x3
Pixel Size: 10.000000 x -10.000000
UL:(699960.000000,7100020.000000)   LR:(809760.000000,6990220.000000)
Copy 0,0,10980,10980 to 0,0,10980,10980.
Copy 0,0,10980,10980 to 0,0,10980,10980.
Copy 0,0,10980,10980 to 0,0,10980,10980.

Processing file     2 of     7, 14.286% completed in 0 minutes.
Filename: /input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20190327T134209_N0207_R124_T21JYL_20190327T202148/T21JYL_20190327T134209_B08_scaled.tif
File Size: 10980x10980x1
Pixel Size: 10.000000 x -10.000000
UL:(699960.000000,7100020.000000)   LR:(809760.000000,6990220.000000)
Copy 0,0,10980,10980 to 0,0,10980,10980.

Processing file     3 of     7, 28.571% completed in 0 minutes.
Filename: /input/SENTINEL2_CACHE/defo

/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/prepared/all_merged_S2B_MSIL1C_20190327T134209_N0207_R124_T21JYK_20190327T202148.tif

all bands are converting to *tif...

TCI
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B08
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B8A
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
B11
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
B12
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.

ndvi band is processing...
0 .. 10 .. 20 .. 30 .. 40 .. 50 .. 60 .. 70 .. 80 .. 90 .. 100 - Done

ndmi band is processing...
0 .. 10 .. 20 .. 30 .. 40 .. 50 .. 60 .. 70 .. 80 .. 90 .. 100 - Done

all bands are scaling to 8-bit images...

/in

Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20200331T134209_N0209_R124_T21JYK_20200331T165744/T21JYK_20200331T134209_B11
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20200331T134209_N0209_R124_T21JYK_20200331T165744/T21JYK_20200331T134209_B12
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20200331T134209_N0209_R124_T21JYK_20200331T165744/ndvi
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
/input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/S2B_MSIL1C_20200331T134209_N0209_R124_T21JYK_20200331T165744/ndmi
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...

In [22]:
start_date_crop_path = crop_aoi_from_raster(
                                    raster_path=start_date_merged_path,
                                    aoi_gdf=aoi,
                                    output_path=os.path.join(PREPARED_DATA_FOLDER,f'prepared_{dates[0]}_date_{tiles_prefix}_crop.tif')
                                    )
end_date_crop_path = crop_aoi_from_raster(
                    raster_path=end_date_merged_path,
                    aoi_gdf=aoi,
                    output_path=os.path.join(PREPARED_DATA_FOLDER,f'prepared_{dates[1]}_date_{tiles_prefix}_crop.tif')
                    )

saved raster /input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/prepared/prepared_20190327_date_21JXK21JYL21JYK_crop.tif
removed raster /input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/prepared/all_merged_20190327_date_21JXK21JYL21JYK_merged.tif
saved raster /input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/prepared/prepared_20200326_date_21JXK21JYL21JYK_crop.tif
removed raster /input/SENTINEL2_CACHE/deforestation_cache_2023_05_25_12:34:37/prepared/all_merged_20200326_date_21JXK21JYL21JYK_merged.tif


### 3.1 Prepare clouds tif files for postprocessing step

In [23]:
from time_dependent.data_prepare.prepare_clouds import (detect_clouds, merge,
                                                        search_band, to_tiff)


def prepare_clouds(data_folder, save_path):
    folder_basename = os.path.basename(data_folder)
    print(folder_basename)
    bands, band_names = [
        "B01",
        "B02",
        "B04",
        "B05",
        "B08",
        "B8A",
        "B09",
        "B10",
        "B11",
        "B12",
    ], []

    for band in bands:
        band_names.append(join(data_folder, search_band(band, data_folder, "jp2")))

    print("\nall bands are converting to *tif...\n")

    for band_name in band_names:
        print(band_name[-3:])
        to_tiff(f"{band_name}.jp2")

    print("\n all bands are being merged...\n")

    save_file_merged = join(save_path, f"{folder_basename}_full_merged.tif")
    print(len(band_names))
    merge(save_file_merged, *band_names)

    save_file_clouds = join(save_path, f"{folder_basename}_clouds.tiff")
    detect_clouds(save_file_merged, save_file_clouds)
    os.remove(save_file_merged)

    for item in os.listdir(data_folder):
        if item.endswith(".tif"):
            os.remove(join(data_folder, item))

    # os.system(f'rm {join(granule_folder, tile_folder, 'IMG_DATA')}*.jp2')
    print("\ntemp files have been deleted\n")
    return save_file_clouds

In [24]:
os.makedirs(CLOUD_DATA_FOLDER, exist_ok=True)

start_clouds = []
end_clouds = []

for start_date_folder, end_date_folder in zip(start_date_folders, end_date_folders):
    start_date_clouds_path = prepare_clouds(start_date_folder, CLOUD_DATA_FOLDER) 
    end_date_clouds_path = prepare_clouds(end_date_folder, CLOUD_DATA_FOLDER)
    
    start_clouds.append(start_date_clouds_path)
    end_clouds.append(end_date_clouds_path)
    
start_date_clouds_path = stitch_tiles(start_clouds, out_raster_path=os.path.join(CLOUD_DATA_FOLDER, f'all_clouds_{dates[0]}_date_{tiles_prefix}.tif'))
end_date_clouds_path = stitch_tiles(end_clouds, out_raster_path=os.path.join(CLOUD_DATA_FOLDER, f'all_clouds_{dates[1]}_date_{tiles_prefix}.tif'))

    
for filename in os.listdir(CLOUD_DATA_FOLDER):
    if not filename.startswith('all_'):
        os.remove(os.path.join(CLOUD_DATA_FOLDER, filename))

S2B_MSIL1C_20190327T134209_N0207_R124_T21JXK_20190327T202148

all bands are converting to *tif...

B01
Input file size is 1830, 1830
0...10...20...30...40...50...60...70...80...90...100 - done.
B02
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B04
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B05
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
B08
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B8A
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
B09
Input file size is 1830, 1830
0...10...20...30...40...50...60...70...80...90...100 - done.
B10
Input file size is 1830, 1830
0...10...20...30...40...50...60...70...80...90...100 - done.
B11
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
B12
Input file size is 5490, 5490
0...10

predict	.
resize.
save cloud.

temp files have been deleted

S2B_MSIL1C_20190327T134209_N0207_R124_T21JYL_20190327T202148

all bands are converting to *tif...

B01
Input file size is 1830, 1830
0...10...20...30...40...50...60...70...80...90...100 - done.
B02
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B04
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B05
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
B08
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B8A
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
B09
Input file size is 1830, 1830
0...10...20...30...40...50...60...70...80...90...100 - done.
B10
Input file size is 1830, 1830
0...10...20...30...40...50...60...70...80...90...100 - done.
B11
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...8

predict	.
resize.
save cloud.

temp files have been deleted

S2B_MSIL1C_20190327T134209_N0207_R124_T21JYK_20190327T202148

all bands are converting to *tif...

B01
Input file size is 1830, 1830
0...10...20...30...40...50...60...70...80...90...100 - done.
B02
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B04
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B05
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
B08
Input file size is 10980, 10980
0...10...20...30...40...50...60...70...80...90...100 - done.
B8A
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.
B09
Input file size is 1830, 1830
0...10...20...30...40...50...60...70...80...90...100 - done.
B10
Input file size is 1830, 1830
0...10...20...30...40...50...60...70...80...90...100 - done.
B11
Input file size is 5490, 5490
0...10...20...30...40...50...60...70...8

predict	.
resize.
save cloud.

temp files have been deleted

saved raster /code/data/clouds/all_clouds_20190327_date_21JXK21JYL21JYK_merged.tif
saved raster /code/data/clouds/all_clouds_20200326_date_21JXK21JYL21JYK_merged.tif


In [25]:
start_date_clouds_crop_path = crop_aoi_from_raster(
                                    raster_path=start_date_clouds_path,
                                    aoi_gdf=aoi,
                                    output_path=os.path.join(CLOUD_DATA_FOLDER,f'clouds_{dates[0]}_date_{tiles_prefix}_crop.tif')
                                    )
end_date_clouds_crop_path = crop_aoi_from_raster(
                    raster_path=end_date_clouds_path,
                    aoi_gdf=aoi,
                    output_path=os.path.join(CLOUD_DATA_FOLDER,f'clouds_{dates[0]}_date_{tiles_prefix}_crop.tif')
                    )

saved raster /code/data/clouds/clouds_20190327_date_21JXK21JYL21JYK_crop.tif
removed raster /code/data/clouds/all_clouds_20190327_date_21JXK21JYL21JYK_merged.tif
saved raster /code/data/clouds/clouds_20190327_date_21JXK21JYL21JYK_crop.tif
removed raster /code/data/clouds/all_clouds_20200326_date_21JXK21JYL21JYK_merged.tif


In [26]:
clouds_files = [start_date_clouds_crop_path, end_date_clouds_crop_path]
clouds_files

['/code/data/clouds/clouds_20190327_date_21JXK21JYL21JYK_crop.tif',
 '/code/data/clouds/clouds_20190327_date_21JXK21JYL21JYK_crop.tif']

### 3.2 Preparing forest landcover data, also needed for postprocessing stage

In [27]:
class LandcoverPolygons:
    """
    LandcoverPolygon class to access forest polygons. Before usage,
    be sure that SENTINEL_TILES file is downloaded.
    SENTINEL_TILES_POLYGONS = 'https://sentinel.esa.int/documents/247904/1955685/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml'

    :param tile: tile name (str), e.g. '36UYA'
    :param crs: coordinate system (str), e.g. 'EPSG:4326'

    :return polygons: list of forest polygons within a tile in CRS of a S2A image
    """

    def __init__(self, tiles, crs, year, aoi):
        self.tiles = tiles
        self.crs = crs

        landcover_tiles = set([tile[:3] for tile in self.tiles])
        self.LANDCOVER_GEOJSON = prepare_landcover(
            year, landcover_tiles, LANDCOVER_POLYGONS_PATH, aoi
        )
        gpd.io.file.fiona.drvsupport.supported_drivers["KML"] = "rw"

    def get_polygon(self):
        polygon_path = os.path.join(LANDCOVER_POLYGONS_PATH, f"{''.join(self.tiles)}.geojson")
        logging.info(f"LANDCOVER_POLYGONS_PATH: {polygon_path}")
        if os.path.isfile(polygon_path):
            logging.info(f"{self.tiles} forests polygons file exists.")
            polygons = gpd.read_file(polygon_path)
        else:
            logging.info(
                f"{self.tiles} forests polygons file does not exist. Creating polygons..."
            )
            polygons = self.create_polygon()

        if len(polygons) > 0:
            polygons = polygons.to_crs(self.crs)
            polygons = list(polygons["geometry"])
        else:
            logging.info("No forests polygons.")
        return polygons

    def create_polygon(self):
        polygons = []
        if os.path.isfile(SENTINEL_TILES):
            logging.info(
                f"read forests_polygons_file: {SENTINEL_TILES}, for tile {self.tiles}"
            )

            sentinel_tiles = gpd.read_file(SENTINEL_TILES, driver="KML")
            sentinel_tiles = sentinel_tiles[sentinel_tiles["Name"].isin(self.tiles)]

            logging.info(f"sentinel_tiles for {self.tiles}: {sentinel_tiles}")

            bounding_polygon = sentinel_tiles["geometry"].values[0]
            polygons = gpd.read_file(self.LANDCOVER_GEOJSON)
            polygons = polygons[polygons["geometry"].intersects(bounding_polygon)]
            polygon_path = os.path.join(LANDCOVER_POLYGONS_PATH, f"{''.join(self.tiles)}.geojson")

            logging.info(f"forests_polygons_file_path: {polygon_path}")

            if polygons.empty:
                return polygons
            polygons.to_file(polygon_path, driver="GeoJSON")
        else:
            logging.error(f"{SENTINEL_TILES} doth not exists")
            raise FileNotFoundError
        return polygons
    

def download_landcover():    
    LANDCOVER_OUTPUT_DIR = os.path.join(SATELLITE_CACHE_FOLDER, "landcover_result")
    os.system("AOI='{0}' START_DATE='{1}' END_DATE='{2}' SENTINEL_CACHE='{3}' OUTPUT_FOLDER='{4}' jupyter nbconvert --inplace --to=notebook --execute ./sip_landcover/landcover.ipynb".format(AOI, START_DATE, END_DATE, SATELLITE_CACHE_FOLDER, LANDCOVER_OUTPUT_DIR))
    
    output_path = os.path.join(LANDCOVER_OUTPUT_DIR, f"{START_DATE}_{END_DATE}.tif")
    return output_path
    
def prepare_landcover(year, landcover_tiles, output_path, aoi):
    filename = f"{''.join(landcover_tiles)}_{year}_landcover"

    for file in os.listdir(output_path):
        if file != LANDCOVER_FILENAME and not file.endswith("_landcover.geojson"):
            os.remove(os.path.join(output_path, file))

#     for file in os.listdir(output_path):
#         if filename in file:
#             return os.path.join(output_path, file)
    try:
        landcover_annual(year, landcover_tiles, output_path, aoi)
        landcover_geojson = create_landcover_gdf(output_path, output_path, filename)
    except Exception:
        raise Exception('Cannot create landcover gdf')
    return landcover_geojson


def landcover_annual(year, landcover_tiles, output_path, aoi):
    # landcover_classes = {
    #    1: "Water",
    #    2: "Trees",
    #    4: "Flooded vegetation",
    #    5: "Crops",
    #    7: "Built Area",
    #    8: "Bare ground",
    #   9: "Snow/Ice",
    #   10: "Clouds",
    #    11: "Rangeland"
    # }
    
    colors_classes = {
        "65,155,223": 1,
        "57,125,73": 2,
        "122,135,198": 4,
        "228,150,53": 5,
        "196,50,27": 7,
        "165,155,143": 8,
        "168,235,255": 9,
        "97,97,97": 10,
        "227,226,195": 11
    }
    
    EPSG = "EPSG:4326"
    landcover_downloaded_path = download_landcover()
    
    with rasterio.open(landcover_downloaded_path) as src:        
        landcover_colored = src.read()
        profile = src.profile
        
        landcover_crop = np.zeros((1, *landcover_colored.shape[-2:]))
        
        for color, landcover_class in colors_classes.items():
            rgb_array = np.array(color.split(','), dtype=np.uint8)
            class_indexes = np.where(np.all(landcover_colored==rgb_array[:, None, None], axis=0))

            landcover_crop[:, class_indexes[0], class_indexes[1]] = landcover_class
        
        profile['count'] = 1
        
        crop_name = os.path.join(
                    output_path, "landcover_crop.tif"
                )
        
        with rasterio.open(crop_name, "w", **profile, nbits=1) as dst:
            dst.write(np.where(landcover_crop == 2, 1, 0).astype(np.uint8))
        
    crops = [crop_name]

    landcover_name = os.path.join(output_path, f"landcover{year}.tif")
    listToStr = " ".join(crops)
    os.system(
        " ".join(
            [
                f"gdalwarp --config GDAL_CACHEMAX 3000 -wm 3000 -t_srs {EPSG}",
                listToStr,
                landcover_name,
            ]
        )
    )
    
    print(f"{landcover_name} was merged")


def rescale(img, ratio):
    width = int(img.shape[1] * ratio)
    height = int(img.shape[0] * ratio)
    dim = (width, height)
    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    return resized


def mask_to_polygons(mask: np.ndarray, transform) -> MultiPolygon:
    """
    Converts raster mask to shapely MultiPolygon
    """
    polygons = []
    shapes = features.shapes(
        mask.astype(np.uint8), mask=(mask > 0), transform=transform
    )

    for shape, _ in shapes:
        polygons.append(shapely.geometry.shape(shape))

    polygons = MultiPolygon(polygons)
    if not polygons.is_valid:
        polygons = polygons.buffer(0)
        if polygons.type == "Polygon":
            polygons = MultiPolygon([polygons])
    return polygons


def create_landcover_gdf(landcover_dir, output_dir, filename):
    landcover_names = [
        name for name in os.listdir(landcover_dir) if name.endswith("_crop.tif")
    ]
    gdfs = []
    for name in tqdm(landcover_names):
        lc_fullpath = os.path.join(landcover_dir, name)
        print(lc_fullpath)
        with rasterio.open(lc_fullpath, "r") as src:
            data = src.read().squeeze()
            data = rescale(data, 0.5)
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
            data = cv2.morphologyEx(data, cv2.MORPH_CLOSE, kernel)
            data = cv2.morphologyEx(data, cv2.MORPH_OPEN, kernel)
            data = rescale(data, 2)

            current_polygons = list(mask_to_polygons(data, src.transform))
            current_polygons = [poly for poly in current_polygons if poly.area]

            current_areas = [poly.area for poly in current_polygons]
            current_tilename = [name] * len(current_areas)

            crs = src.crs
            current_gdf = gpd.GeoDataFrame(
                {
                    "area": current_areas,
                    "names": current_tilename,
                    "geometry": current_polygons,
                },
                crs=crs,
            )
            current_gdf.to_crs("EPSG:4326", inplace=True)
            gdfs.append(current_gdf.copy())

    gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="EPSG:4326")

    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{filename}.geojson")
    gdf.to_file(output_path, driver="GeoJSON")
    return output_path


def weights_exists_or_download(path, file_id):
    if not Path(path).exists():
        creds_file = os.environ.get("CREDENTIAL_FILE")
        creds = service_account.Credentials.from_service_account_file(
            creds_file, scopes=SCOPES
        )

        service = build("drive", "v3", credentials=creds)
        request = service.files().get_media(fileId=file_id)

        fh = io.FileIO("unet_v4.pth", mode="wb")
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}")

    return path

### 4. Unet-diff inference on the prepared data and then postprocessing on cloud and forest landcover data

In [33]:



os.environ.get("CUDA_VISIBLE_DEVICES", "0")

logging.basicConfig(format="%(asctime)s %(message)s")


def predict(image_tensor, model, channels, neighbours, size, device):
    image_shape = 1, count_channels(channels) * neighbours, size, size
    prediction, _ = model.predict(
        image_tensor.view(image_shape).to(device, dtype=torch.float)
    )
    result = prediction.view(size, size).detach().cpu().numpy()
    return result


def diff(img1, img2):
    img2 = match_histograms(img2, img1, multichannel=True)
    difference = (img1 - img2) / (img1 + img2)
    difference = (difference + 1) * 127
    return np.concatenate(
        (difference.astype(np.uint8), img1.astype(np.uint8), img2.astype(np.uint8)),
        axis=-1,
    )


def mask_postprocess(mask):
    kernel = np.ones((3, 3), np.uint8)
    erosion = cv2.erode(mask, kernel, iterations=1)
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(erosion, cv2.MORPH_CLOSE, kernel)
    return closing


def predict_raster(
    img_current,
    img_previous,
    channels,
    network="unet-diff",
    model_weights_path=MODEL_PATH,
    input_size=56,
    neighbours=3,
):
    model, device = load_model(network, model_weights_path)

    with rasterio.open(img_current) as source_current, rasterio.open(
        img_previous
    ) as source_previous:
        meta = source_current.meta
        meta["count"] = 1
        
        height = (source_current.height 
                  if  source_current.height % input_size == 0 
                  else (source_current.height + (input_size - source_current.height % input_size)))
        width = (source_current.width 
                  if  source_current.width % input_size == 0 
                  else (source_current.width + (input_size - source_current.width % input_size)))
        clearcut_mask = np.zeros((height, width))
        for i in tqdm(range(width // input_size)):
            for j in range(height // input_size):
                bottom_row = i * input_size
                upper_row = (i + 1) * input_size
                left_column = j * input_size
                right_column = (j + 1) * input_size

                corners = [
                    source_current.xy(bottom_row, left_column),
                    source_current.xy(bottom_row, right_column),
                    source_current.xy(upper_row, right_column),
                    source_current.xy(upper_row, left_column),
                    source_current.xy(bottom_row, left_column),
                ]

                window = Window(bottom_row, left_column, input_size, input_size)
                image_current = reshape_as_image(source_current.read(window=window))
                image_previous = reshape_as_image(source_previous.read(window=window))
                              
                h, w = image_current.shape[0:2]
                image_current = np.pad(image_current,
                                       pad_width=((0,input_size-h),
                                                  (0,input_size-w),
                                                  (0,0)))
                image_previous = np.pad(image_previous,
                                       pad_width=((0,input_size-h),
                                                  (0,input_size-w),
                                                  (0,0)))

                if np.any(image_current) and np.any(image_previous):
                    difference_image = diff(image_current, image_previous)
                    image_tensor = transforms.ToTensor()(
                        difference_image.astype(np.uint8)
                    ).to(device, dtype=torch.float)

                    predicted = predict(
                        image_tensor, model, channels, neighbours, input_size, device
                    )
                    predicted = mask_postprocess(predicted)
                    clearcut_mask[
                        left_column:right_column, bottom_row:upper_row
                    ] += predicted
                    
    meta["dtype"] = "float32"
    return clearcut_mask.astype(np.float32), meta


def count_channels(channels):
    count = 0
    for ch in channels:
        ch = ch.lower()
        if ch == "rgb":
            count += 3
        elif ch in ["ndvi", "ndmi", "b08", "b8a", "b11", "b12"]:
            count += 1
        else:
            raise Exception("{} channel is unknown!".format(ch))

    return count


def scale(tensor, max_value):
    max_ = tensor.max()
    if max_ > 0:
        return tensor / max_ * max_value
    return tensor


def save_raster(raster_array, meta, save_path, filename):
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)
        logging.info("Data directory created.")

    save_path = os.path.join(save_path, filename)

    with rasterio.open(f"{save_path}.tif", "w", **meta) as dst:
        for i in range(1, meta["count"] + 1):
            dst.write(raster_array, i)



def polygonize(raster_array, meta, transform=True, mode=cv2.RETR_TREE):
    raster_array = (raster_array * 255).astype(np.uint8)

    contours, _ = cv2.findContours(raster_array, mode, cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for i in tqdm(range(len(contours))):
        c = contours[i]
        n_s = (c.shape[0], c.shape[2])
        if n_s[0] > 2:
            if transform:
                polys = [tuple(i) * meta["transform"] for i in c.reshape(n_s)]
            else:
                polys = [tuple(i) for i in c.reshape(n_s)]
            polygons.append(Polygon(polys))

    return polygons

def dump_no_data_geosjon(polygon, geojson_path):
    feature = geojson.Feature(geometry=polygon, properties=dict(label='No data', style=dict(color='red')))
    feature['start_date'] = START_DATE
    feature['end_date'] = END_DATE
    feature['name'] = 'No deforestation found\nNo data available'

    with open(geojson_path, 'w') as f:
        geojson.dump(feature, f)
        

def save_polygons(polygons, 
                  save_path, 
                  filename, 
                  aoi):
    
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)
        logging.info("Data directory created.")
        
    geojson_output_path = os.path.join(save_path, f"{filename}.geojson")
    if len(polygons) == 0:
        logging.info("no_polygons detected")
        dump_no_data_geosjon(aoi.geometry[0], geojson_output_path)
        
    else:
        logging.info(f"{filename} saved.")
        print(f"{filename} saved.")
        polygons['start_date'] = START_DATE
        polygons['end_date'] = END_DATE
        
        polygons = polygons.to_crs("EPSG:4326")
        
        style = {
            "color": "#e80e27",
            "stroke": "#e80e27",
            "stroke-width": 2
        }

        # Assign the style to each polygon using apply()
        polygons["style"] = polygons.apply(lambda row: style, axis=1)
        
        polygons.to_file(geojson_output_path, driver="GeoJSON")


def intersection_poly(test_poly, mask_poly):
    intersecion_score = False
    if test_poly.is_valid and mask_poly.is_valid:
        intersection_result = test_poly.intersection(mask_poly)
        if not intersection_result.is_valid:
            intersection_result = intersection_result.buffer(0)
        if not intersection_result.is_empty:
            intersecion_score = True
    return intersecion_score


def morphological_transform(img):
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    kernel = np.ones((3, 3), np.uint8)
    closing = cv2.dilate(closing, kernel, iterations=1)
    return closing


def postprocessing(tiles, cloud_files, clearcuts, src_crs, year, aoi):
    def get_intersected_polygons(polygons, masks, mask_column_name):
        """Finding in GeoDataFrame with clearcuts the masked polygons.

        :param polygons: GeoDataFrame with clearcuts and mask columns
        :param masks: list of masks (e.g., polygons of clouds)
        :param mask_column_name: name of mask column in polygons GeoDataFrame

        :return: GeoDataFrame with filled mask flags in corresponding column
        """
        masked_values = []
        if len(masks) > 0:
            centroids = [[mask.centroid.x, mask.centroid.y] for mask in masks]
            kdtree = spatial.KDTree(centroids)
            for _, clearcut in polygons.iterrows():
                polygon = clearcut["geometry"]
                _, idxs = kdtree.query(polygon.centroid, k=NEAREST_POLYGONS_NUMBER)
                masked_value = 0
                for idx in idxs:
                    if idx >= len(masks):
                        break
                    if intersection_poly(polygon, masks[idx].buffer(0)):
                        masked_value = 1
                        break
                masked_values.append(masked_value)
        polygons[mask_column_name] = masked_values
        return polygons

    landcover = LandcoverPolygons(tiles, src_crs, year, aoi)
    forest_polygons = landcover.get_polygon()

    #     cloud_files = [f"{img_path}/{tile}_{i}/clouds.tiff" for i in range(DATES_FOR_TILE)]
    cloud_polygons = []
    for cloud_file in cloud_files:
        with rasterio.open(cloud_file) as src:
            clouds = src.read(1)
            meta = src.meta
        clouds = morphological_transform(clouds)
        clouds = (clouds > CLOUDS_PROBABILITY_THRESHOLD).astype(np.uint8)
        if clouds.sum() > 0:
            cloud_polygons.extend(polygonize(clouds, meta, mode=cv2.RETR_LIST))

    n_clearcuts = len(clearcuts)
    polygons = {
        "geometry": gpd.GeoSeries(clearcuts),
        "forest": np.zeros(n_clearcuts),
        "clouds": np.zeros(n_clearcuts),
    }

    polygons = geopandas.GeoDataFrame(polygons, crs=src_crs)

    if len(cloud_polygons) > 0:
        polygons = get_intersected_polygons(polygons, cloud_polygons, "clouds")
    else:
        print("Clouds with specified CLOUDS_PROBABILITY_THRESHOLD not found")
    polygons = get_intersected_polygons(polygons, forest_polygons, "forest")
    polygons = polygons.loc[(polygons.forest == 1) & (polygons.clouds == 0)]
    
    return polygons

In [34]:
def deforestation_inference(
    img_start_path,
    img_end_path,
    tiles,
    network="unet-diff",
    model_weights_path=MODEL_PATH,
    save_path=OUTPUT_FOLDER,
    channels=["RGB", "B08", "B8A", "B11", "B12", "NDVI", "NDMI"],
    threshold=0.4,
    polygonize_only=False,
    clouds_files=clouds_files,
    landcover_aoi=aoi,
):
    predicted_filename = f"predicted_{''.join(dates)}_{tiles_prefix}"

    raster_array, meta = predict_raster(
        img_end_path, img_start_path, channels, network, model_weights_path
    )

    logging.info("Polygonize raster array of clearcuts...")
    clearcuts = polygonize(raster_array > threshold, meta)
    logging.info("Filter polygons of clearcuts")
    polygons = postprocessing(
        tiles,
        clouds_files,
        clearcuts,
        meta["crs"],
        year,
        landcover_aoi,
    )
    
    save_polygons(polygons, 
                  save_path, 
                  predicted_filename, 
                  landcover_aoi)

In [35]:
deforestation_inference(start_date_crop_path, end_date_crop_path, tiles, 
                        clouds_files=clouds_files,
                        landcover_aoi=aoi)

100%|██████████| 95/95 [00:00<00:00, 10908.02it/s]
[NbConvertApp] Converting notebook ./sip_landcover/landcover.ipynb to notebook
[NbConvertApp] Writing 12554 bytes to sip_landcover/landcover.ipynb


Copying color table from /code/data/landcovers/landcover_crop.tif to new file.
Creating output file that is 7322P x 3291L.
Processing input file /code/data/landcovers/landcover_crop.tif.
Using internal nodata values (e.g. 0) for image /code/data/landcovers/landcover_crop.tif.
Copying nodata values from source /code/data/landcovers/landcover_crop.tif to destination /code/data/landcovers/landcover2019.tif.
0...10...20...30...40...50...60...70...80...90...100 - done.
/code/data/landcovers/landcover2019.tif was merged


  0%|          | 0/1 [00:00<?, ?it/s]

/code/data/landcovers/landcover_crop.tif


100%|██████████| 9/9 [00:00<00:00, 7864.32it/s]


predicted_2019032720200326_21JXK21JYL21JYK saved.


#### Delete all intermediate files

In [36]:
shutil.rmtree(SATELLITE_CACHE_FOLDER)